In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/DACON_JudgmentPrediction/data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/DACON_JudgmentPrediction/data/test.csv')

In [ ]:
train.head()

,ID,first_party,second_party,facts,first_party_winner
0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",1
1,TRAIN_0001,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...,0
2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...,1
3,TRAIN_0003,Linkletter,Walker,Victor Linkletter was convicted in state court...,0
4,TRAIN_0004,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud...",1


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [ ]:
test.head()

,ID,first_party,second_party,facts
0,TEST_0000,Salerno,United States,The 1984 Bail Reform Act allowed the federal c...
1,TEST_0001,Milberg Weiss Bershad Hynes and Lerach,"Lexecon, Inc.",Lexecon Inc. was a defendant in a class action...
2,TEST_0002,No. 07-582\t Title: \t Federal Communications ...,"Fox Television Stations, Inc., et al.","In 2002 and 2003, Fox Television Stations broa..."
3,TEST_0003,Harold Kaufman,United States,During his trial for armed robbery of a federa...
4,TEST_0004,Berger,Hanlon,"In 1993, a magistrate judge issued a warrant a..."


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from transformers import AutoTokenizer, BertTokenizer

### 여러가지 방법으로 토큰화를 진행해보았다

In [ ]:
print('단어 토큰화1 :',word_tokenize(train['facts'][0]))


단어 토큰화1 : ['On', 'June', '27', ',', '1962', ',', 'Phil', 'St.', 'Amant', ',', 'a', 'candidate', 'for', 'public', 'office', ',', 'made', 'a', 'television', 'speech', 'in', 'Baton', 'Rouge', ',', 'Louisiana', '.', 'During', 'this', 'speech', ',', 'St.', 'Amant', 'accused', 'his', 'political', 'opponent', 'of', 'being', 'a', 'Communist', 'and', 'of', 'being', 'involved', 'in', 'criminal', 'activities', 'with', 'the', 'head', 'of', 'the', 'local', 'Teamsters', 'Union', '.', 'Finally', ',', 'St.', 'Amant', 'implicated', 'Herman', 'Thompson', ',', 'an', 'East', 'Baton', 'Rouge', 'deputy', 'sheriff', ',', 'in', 'a', 'scheme', 'to', 'move', 'money', 'between', 'the', 'Teamsters', 'Union', 'and', 'St.', 'Amant', '’', 's', 'political', 'opponent', '.', 'Thompson', 'successfully', 'sued', 'St.', 'Amant', 'for', 'defamation', '.', 'Louisiana', '’', 's', 'First', 'Circuit', 'Court', 'of', 'Appeals', 'reversed', ',', 'holding', 'that', 'Thompson', 'did', 'not', 'show', 'St.', 'Amant', 'acted', 'with

1. word_tokenize는 Don't를 Do와 n't로 분리하였으며, 반면 Jone's는 Jone과 's로 분리한 것을 확인할 수 있습니다.

In [ ]:
print('단어 토큰화2 :',WordPunctTokenizer().tokenize(train['facts'][0]))


단어 토큰화2 : ['On', 'June', '27', ',', '1962', ',', 'Phil', 'St', '.', 'Amant', ',', 'a', 'candidate', 'for', 'public', 'office', ',', 'made', 'a', 'television', 'speech', 'in', 'Baton', 'Rouge', ',', 'Louisiana', '.', 'During', 'this', 'speech', ',', 'St', '.', 'Amant', 'accused', 'his', 'political', 'opponent', 'of', 'being', 'a', 'Communist', 'and', 'of', 'being', 'involved', 'in', 'criminal', 'activities', 'with', 'the', 'head', 'of', 'the', 'local', 'Teamsters', 'Union', '.', 'Finally', ',', 'St', '.', 'Amant', 'implicated', 'Herman', 'Thompson', ',', 'an', 'East', 'Baton', 'Rouge', 'deputy', 'sheriff', ',', 'in', 'a', 'scheme', 'to', 'move', 'money', 'between', 'the', 'Teamsters', 'Union', 'and', 'St', '.', 'Amant', '’', 's', 'political', 'opponent', '.', 'Thompson', 'successfully', 'sued', 'St', '.', 'Amant', 'for', 'defamation', '.', 'Louisiana', '’', 's', 'First', 'Circuit', 'Court', 'of', 'Appeals', 'reversed', ',', 'holding', 'that', 'Thompson', 'did', 'not', 'show', 'St', '.',

2. WordPunctTokenizer는 구두점을 별도로 분류하는 특징을 갖고 있기때문에, 앞서 확인했던 word_tokenize와는 달리 Don't를 Don과 '와 t로 분리하였으며, 이와 마찬가지로 Jone's를 Jone과 '와 s로 분리한 것을 확인할 수 있습니다. 

In [ ]:
print('단어 토큰화3 :',text_to_word_sequence(train['facts'][0]))


단어 토큰화3 : ['on', 'june', '27', '1962', 'phil', 'st', 'amant', 'a', 'candidate', 'for', 'public', 'office', 'made', 'a', 'television', 'speech', 'in', 'baton', 'rouge', 'louisiana', 'during', 'this', 'speech', 'st', 'amant', 'accused', 'his', 'political', 'opponent', 'of', 'being', 'a', 'communist', 'and', 'of', 'being', 'involved', 'in', 'criminal', 'activities', 'with', 'the', 'head', 'of', 'the', 'local', 'teamsters', 'union', 'finally', 'st', 'amant', 'implicated', 'herman', 'thompson', 'an', 'east', 'baton', 'rouge', 'deputy', 'sheriff', 'in', 'a', 'scheme', 'to', 'move', 'money', 'between', 'the', 'teamsters', 'union', 'and', 'st', 'amant’s', 'political', 'opponent', 'thompson', 'successfully', 'sued', 'st', 'amant', 'for', 'defamation', 'louisiana’s', 'first', 'circuit', 'court', 'of', 'appeals', 'reversed', 'holding', 'that', 'thompson', 'did', 'not', 'show', 'st', 'amant', 'acted', 'with', '“malice', '”', 'thompson', 'then', 'appealed', 'to', 'the', 'supreme', 'court', 'of', 'l

3. 케라스의 text_to_word_sequence는 기본적으로 모든 알파벳을 소문자로 바꾸면서 마침표나 컴마, 느낌표 등의 구두점을 제거합니다. 하지만 don't나 jone's와 같은 경우 아포스트로피는 보존하는 것을 볼 수 있습니다.

In [ ]:
#문장 유형 분류 경진대회에서 9등한 팀이 이거 씀
auto_tokenizer = AutoTokenizer.from_pretrained('klue/roberta-large')
token = auto_tokenizer.tokenize(train['facts'][0])
print('단어 토큰화4 :',token)


단어 토큰화4 : ['On', 'J', '##une', '27', ',', '1962', ',', 'Phil', 'St', '.', 'Am', '##ant', ',', 'a', 'can', '##d', '##id', '##ate', 'for', 'public', 'of', '##fic', '##e', ',', 'm', '##ade', 'a', 't', '##el', '##ev', '##ision', 'sp', '##ee', '##ch', 'in', 'B', '##at', '##on', 'Ro', '##ug', '##e', ',', 'L', '##ou', '##is', '##ian', '##a', '.', 'D', '##ur', '##ing', 'this', 'sp', '##ee', '##ch', ',', 'St', '.', 'Am', '##ant', 'ac', '##c', '##us', '##ed', 'his', 'pol', '##iti', '##cal', 'op', '##p', '##one', '##nt', 'of', 'be', '##ing', 'a', 'Commun', '##ist', 'and', 'of', 'be', '##ing', 'in', '##v', '##ol', '##ve', '##d', 'in', 'c', '##ri', '##min', '##al', 'ac', '##tiv', '##ities', 'with', 'the', 'he', '##ad', 'of', 'the', 'l', '##oc', '##al', 'Te', '##am', '##st', '##ers', 'Un', '##ion', '.', 'Fin', '##ally', ',', 'St', '.', 'Am', '##ant', 'imp', '##lic', '##ated', 'Her', '##man', 'Thom', '##ps', '##on', ',', 'an', 'East', 'B', '##at', '##on', 'Ro', '##ug', '##e', 'de', '##p', '##uty', 's

In [ ]:
auto_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
token = auto_tokenizer.tokenize(train['facts'][0])
print('단어 토큰화4 :',token)


단어 토큰화4 : ['On', 'June', '27', ',', '1962', ',', 'Phil', 'St', '.', 'Am', '##ant', ',', 'a', 'candidate', 'for', 'public', 'office', ',', 'made', 'a', 'television', 'speech', 'in', 'Baton', 'Rouge', ',', 'Louisiana', '.', 'During', 'this', 'speech', ',', 'St', '.', 'Am', '##ant', 'accused', 'his', 'political', 'opponent', 'of', 'being', 'a', 'Communist', 'and', 'of', 'being', 'involved', 'in', 'criminal', 'activities', 'with', 'the', 'head', 'of', 'the', 'local', 'Teams', '##ters', 'Union', '.', 'Finally', ',', 'St', '.', 'Am', '##ant', 'implicated', 'Herman', 'Thompson', ',', 'an', 'East', 'Baton', 'Rouge', 'deputy', 'sheriff', ',', 'in', 'a', 'scheme', 'to', 'move', 'money', 'between', 'the', 'Teams', '##ters', 'Union', 'and', 'St', '.', 'Am', '##ant', '’', 's', 'political', 'opponent', '.', 'Thompson', 'successfully', 'sued', 'St', '.', 'Am', '##ant', 'for', 'def', '##ama', '##tion', '.', 'Louisiana', '’', 's', 'First', 'Circuit', 'Court', 'of', 'Appeals', 'reversed', ',', 'holding'

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
     

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

In [ ]:
stop_words_list = stopwords.words('english')
print('불용어 개수 :', len(stop_words_list))
print('불용어들 :',stop_words_list)

불용어 개수 : 179
불용어들 : ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'sa

In [ ]:
# 공백으로만 단어를 분류한다.
vectorizer = TfidfVectorizer()

In [ ]:
def get_vector(vectorizer, df, train_mode):
    if train_mode:
        X_facts = vectorizer.fit_transform(df['facts'])
    else:
        X_facts = vectorizer.transform(df['facts'])
    X_party1 = vectorizer.transform(df['first_party'])
    X_party2 = vectorizer.transform(df['second_party'])
    
    X = np.concatenate([X_party1.todense(), X_party2.todense(), X_facts.todense()], axis=1)
    return X

In [ ]:
X_train = get_vector(vectorizer, train, True)
Y_train = train["first_party_winner"]
X_test = get_vector(vectorizer, test, False)

In [ ]:
X_train

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])